In [79]:
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string

In [80]:
# Importing the data

#df = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
df = pd.read_csv('imdb_labelled.txt', sep='  \t', header=None)
df.columns = ['Sentence', 'Score']

<ipython-input-80-ce2da8e7c08e>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('imdb_labelled.txt', sep='  \t', header=None)


In [81]:
#Setting the bools for the score
df.loc[df['Score'] == 0, 'Bool'] = True
df.loc[df['Score'] == 1, 'Bool'] = False

Create a list of words that are in the negative and positive comments.

In [82]:
negative_words = pd.DataFrame([])
positive_words = pd.DataFrame([])
lower_no_punc = pd.DataFrame([])
#[^\w\d\s]+
for index in df.index:
    if df.iloc[index]['Score'] == 0:
        s = df.iloc[index]['Sentence']
        out = re.sub(r'[{}]'.format(string.punctuation), ' ', s)
        negative_words = negative_words.append(out.lower().split())

    if df.iloc[index]['Score'] == 1:
        s = df.iloc[index]['Sentence']
        out = re.sub(r'[{}]'.format(string.punctuation), ' ', s)
        positive_words = positive_words.append(out.lower().split())
    lower_no_punc = lower_no_punc.append([out.lower().strip()])

lower_no_punc = lower_no_punc.reset_index(drop=True)
lower_no_punc.columns = ['Sentence']
negative_words = pd.DataFrame(negative_words[0].unique())
positive_words = pd.DataFrame(positive_words[0].unique())

Dropping out the words that are in both positive and negative lists.

In [83]:
#Compare word lists to each other and remove matching.
neg_match = negative_words[0].isin(positive_words[0])
pos_match = positive_words[0].isin(negative_words[0])

for index in neg_match.index:
    if neg_match.iloc[index] == True:
        negative_words.drop(index, inplace=True)

for index in pos_match.index:
    if pos_match.iloc[index] == True:
        positive_words.drop(index, inplace=True)
        
negative_words = negative_words.reset_index(drop=True)
positive_words = positive_words.reset_index(drop=True)
# Drops anything that is less then three words, reducing match errors.
negative_words.drop(negative_words[negative_words[0].str.len() < 3].index, inplace=True)
positive_words.drop(positive_words[positive_words[0].str.len() < 3].index, inplace=True)

Outputting the result of the word list to see if it matches the score.

In [84]:
# for index in lower_no_punc.index:
#     if any(neg in lower_no_punc.loc[index]['Sentence'] for neg in negative_words[0]):
#         #print(lower_no_punc.loc[index]['Sentence'], any(neg in lower_no_punc.loc[index]['Sentence'] for neg in negative_words[0]))
#         df.loc[index, 'Neg'] = True
for neg in negative_words[0]:
    df[str(neg)] = lower_no_punc['Sentence'].str.contains(str(neg), case=False)


#df['Neg'] = df['Neg'].fillna(False)
df

,Sentence,Score,Bool,slow,aimless,distressed,drifting,sure,flat,nearly,...,fest,spoiled,brat,babysitting,faster,jessice,clothes,virtue,regrettable,exceptionally
0,"A very, very, very slow-moving, aimless movie ...",0,True,True,True,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Not sure who was more lost - the flat characte...,0,True,False,False,False,False,True,True,True,...,False,False,False,False,False,False,False,False,False,False
2,Attempting artiness with black & white and cle...,0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Very little music or anything to speak of.,0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,The best scene in the movie was when Gerardo i...,1,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,I just got bored watching Jessice Lange take h...,0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,False,False,False
996,"Unfortunately, any virtue in this film's produ...",0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,True,False
997,"In a word, it is embarrassing.",0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
998,Exceptionally bad!,0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [ ]:
# print(len(df.loc[(df['Neg'] == False) & (df['Score'] == 0)]))
# print(df.loc[(df['Neg'] == False) & (df['Score'] == 0)])

In [ ]:
# print(len(df.loc[(df['Neg'] == True) & (df['Score'] == 1)]))
# print(df.loc[(df['Neg'] == True) & (df['Score'] == 1)])

In [ ]:
# error = ((len(df.loc[(df['Neg'] == True) & (df['Score'] == 1)])) + len(df.loc[(df['Neg'] == False) & (df['Score'] == 0)])) / len(df)
# print(error * 100, '%')

## Negative List Errors
29% of the sentences are not matching up to the score from the negative word list.  Lets see if the positive word list does much better. 

In [52]:
# for index in lower_no_punc.index:
#     if any(pos in lower_no_punc.loc[index]['Sentence'] for pos in positive_words[0]):
#         #print(lower_no_punc.loc[index]['Sentence'], any(neg in lower_no_punc.loc[index]['Sentence'] for neg in negative_words[0]))
#         df.loc[index, 'Pos'] = True
#     #else:
#         #df.loc[index, 'Neg'] = False
# df['Pos'] = df['Pos'].fillna(False)

In [ ]:
# print(len(df.loc[(df['Pos'] == False) & (df['Score'] == 0)]))
# print(df.loc[(df['Pos'] == False) & (df['Score'] == 0)])

In [ ]:
# print(len(df.loc[(df['Pos'] == True) & (df['Score'] == 1)]))
# print(df.loc[(df['Pos'] == True) & (df['Score'] == 1)])

In [ ]:
# error = ((len(df.loc[(df['Pos'] == True) & (df['Score'] == 1)])) + len(df.loc[(df['Pos'] == False) & (df['Score'] == 0)])) / len(df)
# print(error * 100, '%')

## Positive List Errors
Looks like we are getting at 72% error from the positive word list.  Lets use the negative word list to run a Naive-Bayes.

In [98]:
from sklearn.naive_bayes import BernoulliNB

#Instantiate the model and store in a variable.
bnb = BernoulliNB()
df.dtypes

Sentence         object
Score             int64
Bool             object
slow               bool
aimless            bool
                  ...  
jessice            bool
clothes            bool
virtue             bool
regrettable        bool
exceptionally      bool
Length: 1028, dtype: object

## Naive Bayes

In [100]:
data = df[negative_words[0]]
target = df['Bool']
target = target.astype('bool')

dtype('bool')

In [101]:

bnb.fit(data, target)

y_pred = bnb.predict(data)

print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

Number of mislabeled points out of a total 1000 points : 268
